# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 9:34AM,241381,10,0085842766,ISDIN Corporation,Released
1,Jul 8 2022 9:34AM,241381,10,0085842771,ISDIN Corporation,Released
2,Jul 8 2022 9:34AM,241381,10,0085842770,ISDIN Corporation,Released
3,Jul 8 2022 9:34AM,241381,10,0085842774,ISDIN Corporation,Released
4,Jul 8 2022 9:34AM,241381,10,0085842801,ISDIN Corporation,Released
5,Jul 8 2022 9:34AM,241381,10,0085842820,ISDIN Corporation,Released
6,Jul 8 2022 9:34AM,241381,10,0085842836,ISDIN Corporation,Released
7,Jul 8 2022 9:34AM,241381,10,0085842830,ISDIN Corporation,Released
8,Jul 8 2022 9:34AM,241381,10,0085842838,ISDIN Corporation,Released
9,Jul 8 2022 9:34AM,241381,10,0085842853,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,241380,Released,10
38,241381,Released,39
39,241382,Released,8
40,241383,Released,7
41,241384,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241380,NaN,NaN,10.0
241381,NaN,NaN,39.0
241382,NaN,NaN,8.0
241383,NaN,NaN,7.0
241384,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241288,0.0,0.0,20.0
241297,0.0,0.0,1.0
241298,0.0,0.0,1.0
241306,0.0,0.0,1.0
241319,3.0,36.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241288,0,0,20
241297,0,0,1
241298,0,0,1
241306,0,0,1
241319,3,36,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241288,0,0,20
1,241297,0,0,1
2,241298,0,0,1
3,241306,0,0,1
4,241319,3,36,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241288,,,20
1,241297,,,1
2,241298,,,1
3,241306,,,1
4,241319,3,36,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation
39,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC"
40,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc."
47,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC"
58,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation
66,Jul 8 2022 9:26AM,241380,12,Hush Hush
76,Jul 8 2022 9:25AM,241377,15,"Laser Pharmaceuticals, LLC"
98,Jul 8 2022 9:22AM,241374,10,ISDIN Corporation
156,Jul 8 2022 9:20AM,241376,20,Alumier Labs Inc.
172,Jul 8 2022 9:15AM,241375,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,,,39
1,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC",,,1
2,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc.",,,7
3,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC",,,11
4,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation,,,8
5,Jul 8 2022 9:26AM,241380,12,Hush Hush,,,10
6,Jul 8 2022 9:25AM,241377,15,"Laser Pharmaceuticals, LLC",,,22
7,Jul 8 2022 9:22AM,241374,10,ISDIN Corporation,,,58
8,Jul 8 2022 9:20AM,241376,20,Alumier Labs Inc.,,,16
9,Jul 8 2022 9:15AM,241375,10,Bio-PRF,,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,39,,
1,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC",1,,
2,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc.",7,,
3,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC",11,,
4,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation,8,,
5,Jul 8 2022 9:26AM,241380,12,Hush Hush,10,,
6,Jul 8 2022 9:25AM,241377,15,"Laser Pharmaceuticals, LLC",22,,
7,Jul 8 2022 9:22AM,241374,10,ISDIN Corporation,58,,
8,Jul 8 2022 9:20AM,241376,20,Alumier Labs Inc.,16,,
9,Jul 8 2022 9:15AM,241375,10,Bio-PRF,11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,39,,
1,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC",1,,
2,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc.",7,,
3,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC",11,,
4,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation,8,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,39.0,NaN,NaN
1,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC",1.0,NaN,NaN
2,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc.",7.0,NaN,NaN
3,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC",11.0,NaN,NaN
4,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation,8.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 9:34AM,241381,10,ISDIN Corporation,39.0,0.0,0.0
1,Jul 8 2022 9:31AM,241384,19,"Graystone, LLC",1.0,0.0,0.0
2,Jul 8 2022 9:31AM,241383,15,"Alliance Pharma, Inc.",7.0,0.0,0.0
3,Jul 8 2022 9:29AM,241379,15,"Mizner Bioscience, LLC",11.0,0.0,0.0
4,Jul 8 2022 9:29AM,241382,10,ISDIN Corporation,8.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4585801,296.0,2.0,0.0
102,965249,4.0,0.0,0.0
12,241380,10.0,0.0,0.0
15,724139,40.0,0.0,0.0
16,241373,0.0,1.0,0.0
18,241351,0.0,1.0,0.0
19,482750,32.0,5.0,0.0
20,724032,30.0,44.0,3.0
21,965383,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4585801,296.0,2.0,0.0
1,102,965249,4.0,0.0,0.0
2,12,241380,10.0,0.0,0.0
3,15,724139,40.0,0.0,0.0
4,16,241373,0.0,1.0,0.0
5,18,241351,0.0,1.0,0.0
6,19,482750,32.0,5.0,0.0
7,20,724032,30.0,44.0,3.0
8,21,965383,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,296.0,2.0,0.0
1,102,4.0,0.0,0.0
2,12,10.0,0.0,0.0
3,15,40.0,0.0,0.0
4,16,0.0,1.0,0.0
5,18,0.0,1.0,0.0
6,19,32.0,5.0,0.0
7,20,30.0,44.0,3.0
8,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,296.0
1,102,Released,4.0
2,12,Released,10.0
3,15,Released,40.0
4,16,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,2.0,0.0,0.0,0.0,1.0,1.0,5.0,44.0,1.0
Released,296.0,4.0,10.0,40.0,0.0,0.0,32.0,30.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,2.0,0.0,0.0,0.0,1.0,1.0,5.0,44.0,1.0
2,Released,296.0,4.0,10.0,40.0,0.0,0.0,32.0,30.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,2.0,0.0,0.0,0.0,1.0,1.0,5.0,44.0,1.0
2,Released,296.0,4.0,10.0,40.0,0.0,0.0,32.0,30.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()